In [17]:
from DO_project_input import *

import gurobipy as gb
from gurobipy import GRB

In [18]:
number_of_orders = len(ID_per_order)
number_of_plants = len(cost_per_unit)
number_of_ports = len(costs_per_port)
number_of_days = 2

In [19]:
model = gb.Model('wharehouse_model')

plant_alloc = model.addVars(number_of_orders, number_of_plants, number_of_days, vtype=GRB.BINARY,name="Plant Allocation")        # number of plant j producing order i
port_alloc = model.addVars(number_of_orders, number_of_ports, number_of_days, vtype=GRB.BINARY,name="Port Allocation")

In [20]:
# order on 1 plant
for index_order in range(number_of_orders):
    tot = 0
    for index_day in range(number_of_days):
        tot += gb.quicksum(plant_alloc[index_order, index_plant, index_day] for index_plant in range(number_of_plants))
    model.addConstr(tot==1, name=f'order_{index_order}_to_ONE_plant')

for index_order in range(number_of_orders):
    tot = 0
    for index_day in range(number_of_days):
        tot += gb.quicksum(plant_alloc[index_order, index_port, index_day] for index_port in range(number_of_ports))
    model.addConstr(tot==1, name=f'order_{index_order}_to_ONE_port')

for index_plants in range(number_of_plants):
    for index_day in range(number_of_days):
        model.addConstr(gb.quicksum(plant_alloc[index_orders, index_plants, index_day] for index_orders in range(number_of_orders)) <= daily_order_capacity_per_plant[index_plants], name=f'plant_{index_plants}_order_limit')

for index_order in range(number_of_orders):
    for index_plant in range(number_of_plants):
        for index_port in range(number_of_ports):
            if index_port not in ports_per_plant[index_plant]:
                tot = 0
                for index_day in range(number_of_days):
                    tot += plant_alloc[index_order, index_plant, index_day] + port_alloc[index_order, index_port, index_day]
                model.addConstr(tot <= 1, name=f'order_{index_order}_plant_{index_plant}_not_conn_to_port_{index_port}')
